In [2]:
import matplotlib
print(matplotlib.__version__)

import sys
import decode
import decode.utils
import torch
import torch.nn as nn  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml

print(f"DECODE version: {decode.utils.bookkeeping.decode_state()}")

3.7.5


/home/GaoQF/miniconda3/envs/decode_env2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/GaoQF/miniconda3/envs/decode_env2/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


DECODE version: 0.10.0


In [ ]:
print(dir(decode.generic.emitter.EmitterSet))

['CoordinateOnlyEmitter', 'EmitterSet', 'EmptyEmitterSet', 'Iterable', 'LooseEmitterSet', 'Optional', 'Path', 'RandomEmitterSet', 'Union', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'at_least_one_dim', 'copy', 'decode', 'gutil', 'np', 'same_dim_tensor', 'same_shape_tensor', 'torch', 'tutil', 'warnings']


In [ ]:
main_device_id = 1 
device_ids = [1, 5, 6] 

device = f'cuda:{device_ids[0]}' # 设置主设备
print(f"Main device: {device}, using GPUs: {device_ids}")

threads = 4
worker = 0 

torch.set_num_threads(threads)

if device != 'cpu':
    if not torch.cuda.is_available():
        raise ValueError("CUDA is not available.")

Main device: cuda:1, using GPUs: [1, 5, 6]


In [4]:
param_path = '/home/GaoQF/Downloads_original/2025-06-26_16-47-28_amax/param_run.yaml'
model_path = '/home/GaoQF/Downloads_original/2025-06-26_16-47-28_amax/model_0.pt'

meta = {
    'Camera': {
        'baseline': 200.0, #200.0, #398.6,
        'e_per_adu': 5.0,
        'em_gain': 100, #100, #50.0,
        'spur_noise': 0.0018 # 0.0018 #0.0015  # if you don't know, you can set this to 0.
    }
}

In [5]:
param = decode.utils.param_io.load_params(param_path)
model = decode.neuralfitter.models.model2.parse(param)
model = decode.utils.model_io.LoadSaveModel(model,
                                            input_file=model_path,
                                            output_file=None).load_init(device=device)

Model instantiated.
Model SHA-1 hash: 81f348b97d78e88884cf860817d5582181d010fe
Loaded pretrained model: /home/GaoQF/Downloads_original/2025-06-26_16-47-28_amax/model_0.pt


In [6]:
if len(device_ids) > 1:
    print(f"Wrapping model with DataParallel on {len(device_ids)} GPUs...")
    model = nn.DataParallel(model, device_ids=device_ids)

Wrapping model with DataParallel on 3 GPUs...


In [7]:
model.to(device)

DataParallel(
  (module): model2(
    (unet_shared): UNet2d(
      (encoder): ModuleList(
        (0): Sequential(
          (0): Sequential(
            (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): ELU(alpha=1.0)
            (2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): ELU(alpha=1.0)
          )
          (1): SEBlock(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (fc): Sequential(
              (0): Linear(in_features=48, out_features=3, bias=False)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=3, out_features=48, bias=False)
              (3): Sigmoid()
            )
          )
        )
        (1): Sequential(
          (0): Sequential(
            (0): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): ELU(alpha=1.0)
            (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (

In [8]:
param = decode.utils.param_io.autofill_dict(meta['Camera'], param.to_dict(), mode_missing='include')
param = decode.utils.param_io.RecursiveNamespace(**param)
print(param)

RecursiveNamespace(Camera=RecursiveNamespace(baseline=200, convert2photons=True, e_per_adu=5.0, em_gain=100, px_size=[160.0, 160.0], qe=0.95, read_sigma=58.8, spur_noise=0.0018), CameraPreset=None, Evaluation=RecursiveNamespace(dist_ax=500.0, dist_lat=250.0, dist_vol=None, match_dims=3), Hardware=RecursiveNamespace(device='cuda:5', device_ix=0, device_simulation='cuda:5', num_worker_train=0, torch_multiprocessing_sharing_strategy=None, torch_threads=4, unix_niceness=0), HyperParameter=RecursiveNamespace(arch_param=RecursiveNamespace(activation='ELU', depth=None, depth_bg=None, depth_shared=2, depth_union=2, init_custom=True, initial_features=48, initial_features_bg=None, inter_features=48, norm=None, norm_bg=None, norm_bg_groups=None, norm_groups=None, norm_head=None, norm_head_groups=None, p_dropout=None, pool_mode='StrideConv', recpt_bg=None, skip_gn_level=None, up_mode='upsample', upsample_mode='nearest', use_last_nl=None), architecture='model_2', auto_restart_param=RecursiveNamespa

In [ ]:
frames1 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/STORM_256x256_0001-1-30000.tif')
frames2 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/STORM_256x256_0001-30001-60000.tif')
frames3 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/STORM_256x256_0001-60001-80000.tif')
frames4 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/STORM_256x256_0001-80001-110000.tif')
frames5 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/STORM_256x256_0001-110001-140000.tif')
frames6 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/STORM_256x256_0001-140001-170000.tif')
frames7 = decode.utils.frames_io.load_tif('/data1/ALLData/GaoQF/STORM/data/Actin/Actin1129/supp/STORM_256x256_0001-170001-200000.tif')
frames = torch.cat((frames1, frames2, frames3, frames4, frames5, frames6, frames7), dim=0) 
print(f"Data loaded to CPU. Shape: {frames.shape}")

In [20]:
camera = decode.simulation.camera.Photon2Camera.parse(param)
camera.device = 'cpu'

frame_proc = decode.neuralfitter.utils.processing.TransformSequence([
    decode.neuralfitter.utils.processing.wrap_callable(camera.backward),
    decode.neuralfitter.frame_processing.AutoCenterCrop(8),
    #decode.neuralfitter.frame_processing.Mirror2D(dims=-1),  # WARNING: You might need to comment this line out. see above
    decode.neuralfitter.scale_transform.AmplitudeRescale.parse(param)
])

# determine extent of frame and its dimension after frame_processing
size_procced = decode.neuralfitter.frame_processing.get_frame_extent(frames.unsqueeze(1).size(), frame_proc.forward)  # frame size after processing
frame_extent = ((-0.5, size_procced[-2] - 0.5), (-0.5, size_procced[-1] - 0.5))

# Setup post-processing
# It's a sequence of backscaling, relative to abs. coord conversion and frame2emitter conversion
post_proc = decode.neuralfitter.utils.processing.TransformSequence([

    decode.neuralfitter.scale_transform.InverseParamListRescale.parse(param),

    decode.neuralfitter.coord_transform.Offset2Coordinate(xextent=frame_extent[0],
                                                          yextent=frame_extent[1],
                                                          img_shape=size_procced[-2:]),

    decode.neuralfitter.post_processing.SpatialIntegration(raw_th=0.2,
                                                          xy_unit='px',
                                                          px_size=(160, 160)
                                                          
                                                          )
])

In [25]:
infer = decode.neuralfitter.Infer(model=model, ch_in=param.HyperParameter.channels_in,
                                  frame_proc=frame_proc, post_proc=post_proc,
                                  device=device, batch_size=128, num_workers=0)

emitter = infer.forward(frames[:])

100%|██████████| 1563/1563 [20:03<00:00,  1.30it/s]


In [26]:
print(emitter)

EmitterSet
::num emitters: 84034315
::xy unit: px
::px size: tensor([160., 160.])
::frame range: 0 - 199999
::spanned volume: [-6.5001690e-01 -5.6254148e-01 -7.3257764e+02] - [255.57579 255.73363 716.43506]


In [27]:
sigma_x_high_threshold = 40 #40
sigma_y_high_threshold = 40 #40
sigma_z_high_threshold = 80 #80

em_sub = emitter[
    (emitter.xyz_sig_nm[:, 0] <= sigma_x_high_threshold)
    * (emitter.xyz_sig_nm[:, 1] <= sigma_x_high_threshold)
    * (emitter.xyz_sig_nm[:, 2] <= sigma_z_high_threshold)
    ]
print(em_sub)

EmitterSet
::num emitters: 65317072
::xy unit: px
::px size: tensor([160., 160.])
::frame range: 0 - 199999
::spanned volume: [-6.5001690e-01 -5.5173343e-01 -7.1696637e+02] - [255.57579 255.73363 716.43506]


In [ ]:
decode.utils.emitter_io.save_csv('/data1/ALLData/GaoQF/STORM/output_csv/Actin1129/0-20w/Actin1129_model9_0-20w.csv', em_sub.data, em_sub.meta)